In [130]:
from fuzzywuzzy import fuzz
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import json
import ast
import time
import numpy as np
import pickle

In [2]:
credits = pd.read_csv('credits.csv')
print(credits.shape)
credits.head()

(45476, 3)


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [3]:
movies = pd.read_csv('movies_metadata.csv')
movies = movies[['adult', 'budget', 'genres', 'id', 'original_language', 'original_title', 'popularity', 'release_date', 'revenue', 'runtime', 'vote_average', 'vote_count']]

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
genres_vector = movies.genres.values
genres = []
for i in range(genres_vector.shape[0]):
    try:
        genres.append('|'.join([k['name'] for k in json.loads(json.dumps(ast.literal_eval(genres_vector[i])))]))
    except:
        genres.append(None)
        
movies.genres = genres

In [5]:
actor1, actor2, actor3, director = [], [], [], []
credits_vector = credits.values
for i in range(credits_vector.shape[0]):
    x=json.loads(json.dumps(ast.literal_eval(credits_vector[i, 0])))
    try:
        found = False
        for k in json.loads(json.dumps(ast.literal_eval(credits_vector[i, 1]))):
            if k['job'] == 'Director':
                director.append(k['name'])
                found = True
                break
        if not found:
            director.append(None)
    except:
        director.append(None)
    try:
        actor1.append(x[0]['name'])
    except:
        actor1.append(None)
    try:
        actor2.append(x[1]['name'])
    except:
        actor2.append(None)
    try:
        actor3.append(x[2]['name'])
    except:
        actor3.append(None)

In [6]:
credits['actor_1_name'] = actor1
credits['actor_2_name'] = actor2
credits['actor_3_name'] = actor3
credits['director_name'] = director
credits = credits[['id', 'actor_1_name','actor_2_name','actor_3_name','director_name']]
keywords = pd.read_csv('keywords.csv')
keywords_vector = keywords.values
plot_keywords = []
for i in range(keywords_vector.shape[0]):
    plot_keywords.append('|'.join([k['name'] for k in json.loads(json.dumps(ast.literal_eval(keywords_vector[i, 1])))]))
keywords['plot_keywords'] = plot_keywords
keywords = keywords[['id', 'plot_keywords']]
keywords.id = keywords.id.astype('str')
movies.id =movies.id.astype('str')
credits.id =credits.id.astype('str')
movies=movies.merge(keywords, on='id')
movies=movies.merge(credits, on='id')
movies[movies.id.apply(lambda x: x.isnumeric())]
movies.id=movies.id.astype('int')
movies.popularity = movies.popularity.astype('str')
def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False
movies[movies.popularity.apply(lambda x: isfloat(x))]
movies.popularity=movies.popularity.astype('float')
movies.release_date = pd.to_datetime(movies.release_date).dt.year
movies[movies.release_date.apply(lambda x: isfloat(x))]
movies.release_date=movies.release_date.astype('float')

In [7]:
print(movies.shape)
movies.head()

(46628, 17)


,adult,budget,genres,id,original_language,original_title,popularity,release_date,revenue,runtime,vote_average,vote_count,plot_keywords,actor_1_name,actor_2_name,actor_3_name,director_name
0,False,30000000,Animation|Comedy|Family,862,en,Toy Story,21.946943,1995.0,373554033.0,81.0,7.7,5415.0,jealousy|toy|boy|friendship|friends|rivalry|bo...,Tom Hanks,Tim Allen,Don Rickles,John Lasseter
1,False,65000000,Adventure|Fantasy|Family,8844,en,Jumanji,17.015539,1995.0,262797249.0,104.0,6.9,2413.0,board game|disappearance|based on children's b...,Robin Williams,Jonathan Hyde,Kirsten Dunst,Joe Johnston
2,False,0,Romance|Comedy,15602,en,Grumpier Old Men,11.712900,1995.0,0.0,101.0,6.5,92.0,fishing|best friend|duringcreditsstinger|old men,Walter Matthau,Jack Lemmon,Ann-Margret,Howard Deutch
3,False,16000000,Comedy|Drama|Romance,31357,en,Waiting to Exhale,3.859495,1995.0,81452156.0,127.0,6.1,34.0,based on novel|interracial relationship|single...,Whitney Houston,Angela Bassett,Loretta Devine,Forest Whitaker
4,False,0,Comedy,11862,en,Father of the Bride Part II,8.387519,1995.0,76578911.0,106.0,5.7,173.0,baby|midlife crisis|confidence|aging|daughter|...,Steve Martin,Diane Keaton,Martin Short,Charles Shyer


In [8]:
list_genres=list(filter(None, list(set('|'.join(movies.genres.values.tolist()).split('|')))))
list_genres

['Horror',
 'War',
 'Foreign',
 'Music',
 'Science Fiction',
 'Animation',
 'Western',
 'Comedy',
 'Thriller',
 'Fantasy',
 'Romance',
 'Documentary',
 'Mystery',
 'Drama',
 'Crime',
 'Action',
 'History',
 'Family',
 'TV Movie',
 'Adventure']

![alt text](CodeCogsEqn.png)

In [9]:
gaussian_filter_2d = lambda x,y,sigma: np.exp((-x**2 -y**2)/(2 * sigma ** 2))

In [133]:
def dynamic_dimension(chosen_index): 
    labels = []
    if movies.director_name.iloc[chosen_index]:
        labels += [movies.director_name.iloc[chosen_index]]
    if movies.actor_1_name.iloc[chosen_index]:
        labels += [movies.actor_1_name.iloc[chosen_index]]
    if movies.actor_2_name.iloc[chosen_index]:
        labels += [movies.actor_1_name.iloc[chosen_index]]
    if movies.actor_3_name.iloc[chosen_index]:
        labels += [movies.actor_3_name.iloc[chosen_index]]
    if movies.original_language.iloc[chosen_index]:
        labels += [movies.original_language.iloc[chosen_index]]
    if movies.plot_keywords.iloc[chosen_index]:
        labels += movies.plot_keywords.iloc[chosen_index].split('|')
    return labels

In [134]:
def add_dimension(df, variables, columns):
    shape_len = df.shape[1]
    df = np.hstack([df, np.zeros((df.shape[0], len(variables)))])
    for i in columns:
        for k in range(df.shape[0]):
            if not df[k,i]:
                continue
            try:
                for s in df[k,i].split('|'):
                    if s in variables:
                        df[k,shape_len+variables.index(s)] += 1
            except Exception as e:
                continue
    return df

In [135]:
def recommend(chosen_index, count_neigh):
    df_copy = movies.copy().values
    variables = list_genres + dynamic_dimension(chosen_index)
    df_copy = add_dimension(df_copy, variables, [4,12,13,14,15,16])
    df_copy = df_copy[:,movies.shape[1]:].astype('float')
    df_copy = df_copy[~np.isnan(df_copy).any(axis=1)]
    neighbors = NearestNeighbors(n_neighbors=count_neigh+1, algorithm='auto', metric='sqeuclidean').fit(df_copy)
    xtest = df_copy[chosen_index, :]
    xtest = xtest.reshape(1, -1)
    distances, indices = neighbors.kneighbors(xtest)
    results = indices[0][:].tolist()
    try:
        del results[results.index(chosen_index)]
    except:
        results=results[:count_neigh]
        pass
    return results

In [136]:
ironman_index = np.argmax([fuzz.ratio('Iron Man', i) for i in movies.original_title.values])
suggest_number = 10
print('ironman selected:')
print(movies.iloc[ironman_index,:])
recommended = recommend(ironman_index, suggest_number)
movies.iloc[recommended,:]

ironman selected:
adult                                                            False
budget                                                       140000000
genres                                Action|Science Fiction|Adventure
id                                                                1726
original_language                                                   en
original_title                                                Iron Man
popularity                                                     22.0731
release_date                                                      2008
revenue                                                    5.85174e+08
runtime                                                            126
vote_average                                                       7.4
vote_count                                                        8951
plot_keywords        middle east|arms dealer|malibu|marvel comic|su...
actor_1_name                                         Robert

,adult,budget,genres,id,original_language,original_title,popularity,release_date,revenue,runtime,vote_average,vote_count,plot_keywords,actor_1_name,actor_2_name,actor_3_name,director_name
21033,False,200000000,Action|Adventure|Science Fiction,68721,en,Iron Man 3,23.721243,2013.0,1.215440e+09,130.0,6.8,8951.0,terrorist|war on terror|tennessee|malibu|marve...,Robert Downey Jr.,Gwyneth Paltrow,Don Cheadle,Shane Black
15324,False,200000000,Adventure|Action|Science Fiction,10138,en,Iron Man 2,19.083344,2010.0,6.239333e+08,124.0,6.6,6969.0,malibu|marvel comic|superhero|based on comic|r...,Robert Downey Jr.,Gwyneth Paltrow,Don Cheadle,Jon Favreau
26782,False,250000000,Adventure|Action|Science Fiction,271110,en,Captain America: Civil War,145.882135,2016.0,1.153304e+09,147.0,7.1,7462.0,civil war|war|marvel comic|sequel|superhero|ba...,Chris Evans,Robert Downey Jr.,Scarlett Johansson,Anthony Russo
18008,False,220000000,Science Fiction|Action|Adventure,24428,en,The Avengers,89.887648,2012.0,1.519558e+09,143.0,7.4,12000.0,new york|shield|marvel comic|superhero|based o...,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Joss Whedon
26777,False,130000000,Science Fiction|Action|Adventure,102899,en,Ant-Man,26.882388,2015.0,5.193120e+08,117.0,7.0,6029.0,marvel comic|superhero|based on comic|aftercre...,Paul Rudd,Michael Douglas,Evangeline Lilly,Peyton Reed
23249,False,170000000,Action|Adventure|Science Fiction,100402,en,Captain America: The Winter Soldier,18.717704,2014.0,7.147666e+08,136.0,7.6,5881.0,washington d.c.|future|shield|marvel comic|sup...,Chris Evans,Samuel L. Jackson,Scarlett Johansson,Anthony Russo
12808,False,150000000,Science Fiction|Action|Adventure,1724,en,The Incredible Hulk,19.125537,2008.0,1.637121e+08,114.0,6.1,3086.0,new york|rio de janeiro|marvel comic|superhero...,Edward Norton,Liv Tyler,Tim Roth,Louis Leterrier
17634,False,140000000,Action|Adventure|Science Fiction,1771,en,Captain America: The First Avenger,19.323582,2011.0,3.705698e+08,124.0,6.6,7174.0,new york|usa|world war ii|nazis|marvel comic|s...,Chris Evans,Hugo Weaving,Tommy Lee Jones,Joe Johnston
17157,False,150000000,Adventure|Fantasy|Action,10195,en,Thor,29.158489,2011.0,4.493266e+08,115.0,6.6,6678.0,new mexico|banishment|shield|marvel comic|hamm...,Chris Hemsworth,Natalie Portman,Tom Hiddleston,Kenneth Branagh
26773,False,280000000,Action|Adventure|Science Fiction,99861,en,Avengers: Age of Ultron,37.379420,2015.0,1.405404e+09,141.0,7.3,6908.0,marvel comic|sequel|superhero|based on comic|v...,Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Joss Whedon


In [137]:
def sequel(title_1, title_2):    
    if fuzz.ratio(title_1, title_2) > 70 or fuzz.token_set_ratio(title_1, title_2) > 70:
        return True
    else:
        return False

def sort_distribution(title_main, std_count, std_date, title, vote, count, date, popularity):
    if vote:
        factor_e_left = gaussian_filter_2d(vote, count, std_count)
    else:
        factor_e_left = -999999999
    if sequel(title_main, title):
        gaussian_left = 0
    else:
        gaussian_left = (1 / (np.pi * 2 * std_count ** 2)) * factor_e_left
    
    if popularity:
        factor_e_right = gaussian_filter_2d(popularity, date, std_date)
    else:
        factor_e_right = -999999999
    gaussian_right = (1 / (np.pi * 2 * std_date ** 2)) * factor_e_right
    return (gaussian_left + gaussian_right) / 2

In [138]:
def extract_ratings(chosen_index, list_films, count_neigh):
    parametres_movie = ['_' for _ in range(count_neigh)]
    counts, dates = [],[]
    for i, index in enumerate(list_films):
        parametres_movie[i] = movies[['original_title','vote_average','vote_count','release_date','popularity']].iloc[index].tolist()
        parametres_movie[i].append(index)
        counts.append(parametres_movie[i][2])
        dates.append(parametres_movie[i][3])
    
    title_main = movies['original_title'].iloc[chosen_index]
    counts.append(movies['vote_count'].iloc[chosen_index])
    dates.append(movies['release_date'].iloc[chosen_index])
    std_counts = np.std(counts)
    std_dates = np.std(dates)
    results_gaussian = [sort_distribution(title_main,std_counts,std_dates,x[0],x[1],x[2],x[3],x[4]) for x in parametres_movie]
    results = np.array(parametres_movie)[(np.array(results_gaussian).argsort())]
    return results[:,-1].tolist()

In [139]:
ironman_index = np.argmax([fuzz.ratio('Iron Man', i) for i in movies.original_title.values])
suggest_number = 10
print('ironman selected:')
print(movies.iloc[ironman_index,:])
recommended = recommend(ironman_index, suggest_number)
sorted_recommended = np.array(extract_ratings(ironman_index, recommended, suggest_number)).astype('int')
movies.iloc[sorted_recommended,:]

ironman selected:
adult                                                            False
budget                                                       140000000
genres                                Action|Science Fiction|Adventure
id                                                                1726
original_language                                                   en
original_title                                                Iron Man
popularity                                                     22.0731
release_date                                                      2008
revenue                                                    5.85174e+08
runtime                                                            126
vote_average                                                       7.4
vote_count                                                        8951
plot_keywords        middle east|arms dealer|malibu|marvel comic|su...
actor_1_name                                         Robert

,adult,budget,genres,id,original_language,original_title,popularity,release_date,revenue,runtime,vote_average,vote_count,plot_keywords,actor_1_name,actor_2_name,actor_3_name,director_name
21033,False,200000000,Action|Adventure|Science Fiction,68721,en,Iron Man 3,23.721243,2013.0,1.215440e+09,130.0,6.8,8951.0,terrorist|war on terror|tennessee|malibu|marve...,Robert Downey Jr.,Gwyneth Paltrow,Don Cheadle,Shane Black
15324,False,200000000,Adventure|Action|Science Fiction,10138,en,Iron Man 2,19.083344,2010.0,6.239333e+08,124.0,6.6,6969.0,malibu|marvel comic|superhero|based on comic|r...,Robert Downey Jr.,Gwyneth Paltrow,Don Cheadle,Jon Favreau
18008,False,220000000,Science Fiction|Action|Adventure,24428,en,The Avengers,89.887648,2012.0,1.519558e+09,143.0,7.4,12000.0,new york|shield|marvel comic|superhero|based o...,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Joss Whedon
26782,False,250000000,Adventure|Action|Science Fiction,271110,en,Captain America: Civil War,145.882135,2016.0,1.153304e+09,147.0,7.1,7462.0,civil war|war|marvel comic|sequel|superhero|ba...,Chris Evans,Robert Downey Jr.,Scarlett Johansson,Anthony Russo
17634,False,140000000,Action|Adventure|Science Fiction,1771,en,Captain America: The First Avenger,19.323582,2011.0,3.705698e+08,124.0,6.6,7174.0,new york|usa|world war ii|nazis|marvel comic|s...,Chris Evans,Hugo Weaving,Tommy Lee Jones,Joe Johnston
26773,False,280000000,Action|Adventure|Science Fiction,99861,en,Avengers: Age of Ultron,37.379420,2015.0,1.405404e+09,141.0,7.3,6908.0,marvel comic|sequel|superhero|based on comic|v...,Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Joss Whedon
17157,False,150000000,Adventure|Fantasy|Action,10195,en,Thor,29.158489,2011.0,4.493266e+08,115.0,6.6,6678.0,new mexico|banishment|shield|marvel comic|hamm...,Chris Hemsworth,Natalie Portman,Tom Hiddleston,Kenneth Branagh
26777,False,130000000,Science Fiction|Action|Adventure,102899,en,Ant-Man,26.882388,2015.0,5.193120e+08,117.0,7.0,6029.0,marvel comic|superhero|based on comic|aftercre...,Paul Rudd,Michael Douglas,Evangeline Lilly,Peyton Reed
23249,False,170000000,Action|Adventure|Science Fiction,100402,en,Captain America: The Winter Soldier,18.717704,2014.0,7.147666e+08,136.0,7.6,5881.0,washington d.c.|future|shield|marvel comic|sup...,Chris Evans,Samuel L. Jackson,Scarlett Johansson,Anthony Russo
12808,False,150000000,Science Fiction|Action|Adventure,1724,en,The Incredible Hulk,19.125537,2008.0,1.637121e+08,114.0,6.1,3086.0,new york|rio de janeiro|marvel comic|superhero...,Edward Norton,Liv Tyler,Tim Roth,Louis Leterrier


## remove sequel

In [140]:
outputs, i = [], 0
while len(outputs) < suggest_number:
    recommended = recommend(ironman_index, suggest_number * (i+2))
    if not sequel(movies['original_title'].iloc[ironman_index], movies['original_title'].iloc[recommended[i]]) and recommended[i] not in outputs:
        outputs.append(recommended[i])
    i += 1
sorted_recommended = np.array(extract_ratings(ironman_index,outputs,suggest_number)).astype('int')

In [141]:
movies.iloc[sorted_recommended,:]

,adult,budget,genres,id,original_language,original_title,popularity,release_date,revenue,runtime,vote_average,vote_count,plot_keywords,actor_1_name,actor_2_name,actor_3_name,director_name
18008,False,220000000,Science Fiction|Action|Adventure,24428,en,The Avengers,89.887648,2012.0,1.519558e+09,143.0,7.4,12000.0,new york|shield|marvel comic|superhero|based o...,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Joss Whedon
26782,False,250000000,Adventure|Action|Science Fiction,271110,en,Captain America: Civil War,145.882135,2016.0,1.153304e+09,147.0,7.1,7462.0,civil war|war|marvel comic|sequel|superhero|ba...,Chris Evans,Robert Downey Jr.,Scarlett Johansson,Anthony Russo
17634,False,140000000,Action|Adventure|Science Fiction,1771,en,Captain America: The First Avenger,19.323582,2011.0,3.705698e+08,124.0,6.6,7174.0,new york|usa|world war ii|nazis|marvel comic|s...,Chris Evans,Hugo Weaving,Tommy Lee Jones,Joe Johnston
26773,False,280000000,Action|Adventure|Science Fiction,99861,en,Avengers: Age of Ultron,37.379420,2015.0,1.405404e+09,141.0,7.3,6908.0,marvel comic|sequel|superhero|based on comic|v...,Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Joss Whedon
17157,False,150000000,Adventure|Fantasy|Action,10195,en,Thor,29.158489,2011.0,4.493266e+08,115.0,6.6,6678.0,new mexico|banishment|shield|marvel comic|hamm...,Chris Hemsworth,Natalie Portman,Tom Hiddleston,Kenneth Branagh
26777,False,130000000,Science Fiction|Action|Adventure,102899,en,Ant-Man,26.882388,2015.0,5.193120e+08,117.0,7.0,6029.0,marvel comic|superhero|based on comic|aftercre...,Paul Rudd,Michael Douglas,Evangeline Lilly,Peyton Reed
23249,False,170000000,Action|Adventure|Science Fiction,100402,en,Captain America: The Winter Soldier,18.717704,2014.0,7.147666e+08,136.0,7.6,5881.0,washington d.c.|future|shield|marvel comic|sup...,Chris Evans,Samuel L. Jackson,Scarlett Johansson,Anthony Russo
26783,False,165000000,Action|Adventure|Fantasy|Science Fiction,284052,en,Doctor Strange,43.847654,2016.0,6.777184e+08,115.0,7.1,5880.0,magic|marvel comic|superhero|based on comic|so...,Benedict Cumberbatch,Chiwetel Ejiofor,Rachel McAdams,Scott Derrickson
22139,False,170000000,Action|Adventure|Fantasy,76338,en,Thor: The Dark World,34.905447,2013.0,6.445714e+08,112.0,6.8,4873.0,marvel comic|superhero|based on comic|hostile ...,Chris Hemsworth,Natalie Portman,Tom Hiddleston,Alan Taylor
42411,False,500000,Adventure|Fantasy|Science Fiction|Action,76122,en,Marvel One-Shot: The Consultant,4.964486,2011.0,0.000000e+00,4.0,6.6,108.0,marvel comic|tony stark|iron man|hulk|marvel c...,Clark Gregg,Maximiliano Hernández,Tim Roth,Leythum


In [142]:
def suggest_me_movies(keyword, suggest_count, remove_sequel=True):
    lasttime = time.time()
    your_index = np.argmax([fuzz.ratio(keyword, i) for i in movies['original_title'].values])
    print('your selection:')
    print(movies.iloc[your_index,:])
    if remove_sequel:
        outputs, i = [], 0
        while len(outputs) < suggest_count:
            recommended = recommend(your_index, suggest_count * (i+2))
            if not sequel(movies['original_title'].iloc[your_index], movies['original_title'].iloc[recommended[i]]) and recommended[i] not in outputs:
                outputs.append(recommended[i])
            i += 1
        results = np.array(extract_ratings(your_index,outputs,suggest_count)).astype('int')
    else:
        recommended = recommend(your_index, suggest_count)
        results = np.array(extract_ratings(your_index,recommended,suggest_count)).astype('int')
    print('time taken to suggest:', time.time()-lasttime, 'seconds')
    return results

In [143]:
movies.iloc[suggest_me_movies('cinderella', 15, remove_sequel=True),:]

your selection:
adult                                                            False
budget                                                         2900000
genres                                Family|Fantasy|Animation|Romance
id                                                               11224
original_language                                                   en
original_title                                              Cinderella
popularity                                                     14.4369
release_date                                                      1950
revenue                                                    2.63591e+08
runtime                                                             74
vote_average                                                       6.8
vote_count                                                        1760
plot_keywords        based on novel|cinderella|loss of parents|mous...
actor_1_name                                               Il

,adult,budget,genres,id,original_language,original_title,popularity,release_date,revenue,runtime,vote_average,vote_count,plot_keywords,actor_1_name,actor_2_name,actor_3_name,director_name
581,False,28000000,Animation|Family|Comedy|Adventure|Fantasy|Romance,812,en,Aladdin,16.357419,1992.0,504050219.0,90.0,7.4,3495.0,magic|musical|cartoon|princess|love|comedy|ani...,Scott Weinger,Robin Williams,Linda Larkin,Ron Clements
588,False,25000000,Romance|Family|Animation|Fantasy|Music,10020,en,Beauty and the Beast,23.433511,1991.0,377350553.0,84.0,7.5,3029.0,prince|book|castle|rose|musical|princess|anima...,Paige O'Hara,Robby Benson,Richard White,Gary Trousdale
1818,False,90000000,Animation|Family|Adventure,10674,en,Mulan,21.180857,1998.0,304320254.0,88.0,7.6,2089.0,homeland|musical|training|daughter|cricket|pri...,Ming-Na Wen,Lea Salonga,Eddie Murphy,Tony Bancroft
2007,False,6000000,Fantasy|Animation|Romance|Family,10882,en,Sleeping Beauty,14.276169,1959.0,51000000.0,75.0,6.8,1332.0,sleep|magic|prince|castle|fairy tale|kingdom|m...,Mary Costa,Bill Shirley,Eleanor Audley,Clyde Geronimi
770,False,100000000,Drama|Animation|Family,10545,en,The Hunchback of Notre Dame,13.553999,1996.0,100138851.0,91.0,6.8,1182.0,paris|based on novel|judge|obsession|dance|swo...,Tom Hulce,Demi Moore,Tony Jay,Gary Trousdale
7296,False,31000000,Family|Fantasy|Comedy,14442,en,Ella Enchanted,11.316988,2004.0,27388767.0,96.0,5.9,454.0,elves|based on novel|magic|fairy|prince|fairy ...,Anne Hathaway,Hugh Dancy,Cary Elwes,Tommy O'Haver
2036,False,26000000,Drama|Romance|Comedy,9454,en,EverAfter,13.309701,1998.0,65705772.0,121.0,6.8,408.0,france|gypsy|child abuse|slavery|leonardo da v...,Drew Barrymore,Dougray Scott,Anjelica Huston,Andy Tennant
4295,False,0,Drama|Romance,17612,en,Lost and Delirious,6.236798,2001.0,0.0,103.0,6.8,132.0,sex|based on novel|roommate|identity|secret|nu...,Piper Perabo,Jessica Paré,Mischa Barton,Léa Pool
11604,False,47000000,Animation|Comedy|Family|Fantasy|Science Fiction,5393,en,Happily N'Ever After,13.139893,2006.0,38085778.0,75.0,4.6,108.0,dwarves|cinderella|wolf|bad mother-in-law|prin...,George Carlin,Andy Dick,Sarah Michelle Gellar,Yvette Kaplan
8943,False,0,Drama|Family,17663,en,Anne of Green Gables,7.222048,1985.0,0.0,199.0,8.2,71.0,based on novel|brother sister relationship|can...,Megan Follows,Colleen Dewhurst,Richard Farnsworth,Kevin Sullivan


In [144]:
movies.iloc[suggest_me_movies('cinderella', 15, remove_sequel=False),:]

your selection:
adult                                                            False
budget                                                         2900000
genres                                Family|Fantasy|Animation|Romance
id                                                               11224
original_language                                                   en
original_title                                              Cinderella
popularity                                                     14.4369
release_date                                                      1950
revenue                                                    2.63591e+08
runtime                                                             74
vote_average                                                       6.8
vote_count                                                        1760
plot_keywords        based on novel|cinderella|loss of parents|mous...
actor_1_name                                               Il

,adult,budget,genres,id,original_language,original_title,popularity,release_date,revenue,runtime,vote_average,vote_count,plot_keywords,actor_1_name,actor_2_name,actor_3_name,director_name
19518,False,5000000,Family|Animation|Romance|Fantasy,14128,en,Cinderella II: Dreams Come True,8.002712,2002.0,0.0,74.0,5.5,265.0,cinderella|magic|fairy|musical|princess|sequel...,Jennifer Hale,Rob Paulsen,Corey Burton,John Kafka
29178,False,8000000,Animation|Romance|Family|Fantasy,16119,en,Cinderella III: A Twist in Time,6.961912,2007.0,0.0,74.0,6.0,244.0,cinderella|magic|fairy|prince|musical|sequel|f...,Jennifer Hale,Susanne Blakeslee,Tress MacNeille,Frank Nissen
28878,False,95000000,Romance|Fantasy|Family|Drama,150689,en,Cinderella,19.801128,2015.0,543514353.0,105.0,6.7,2426.0,cinderella|magic|prince|fairy tale|kingdom|roy...,Lily James,Cate Blanchett,Richard Madden,Kenneth Branagh
42032,False,0,Comedy|Family|Fantasy|Music,407655,en,A Cinderella Story: If the Shoe Fits,5.761235,2016.0,0.0,92.0,6.1,99.0,dancer|cinderella|dance|fairy tale|musical|cel...,Sofia Carson,Thomas Law,Jennifer Tilly,Michelle Johnston
18382,False,5000000,Music,15157,en,Another Cinderella Story,7.863793,2008.0,0.0,90.0,6.2,406.0,dancer|cinderella|dance|masked ball|fairy tale...,Selena Gomez,Jane Lynch,Katharine Isabelle,Damon Santostefano
581,False,28000000,Animation|Family|Comedy|Adventure|Fantasy|Romance,812,en,Aladdin,16.357419,1992.0,504050219.0,90.0,7.4,3495.0,magic|musical|cartoon|princess|love|comedy|ani...,Scott Weinger,Robin Williams,Linda Larkin,Ron Clements
588,False,25000000,Romance|Family|Animation|Fantasy|Music,10020,en,Beauty and the Beast,23.433511,1991.0,377350553.0,84.0,7.5,3029.0,prince|book|castle|rose|musical|princess|anima...,Paige O'Hara,Robby Benson,Richard White,Gary Trousdale
25704,False,50000000,Fantasy|Comedy|Music,224141,en,Into the Woods,9.580785,2014.0,212902372.0,125.0,5.6,1688.0,witch|cinderella|prince|fairy tale|musical|pri...,Meryl Streep,Emily Blunt,James Corden,Rob Marshall
2007,False,6000000,Fantasy|Animation|Romance|Family,10882,en,Sleeping Beauty,14.276169,1959.0,51000000.0,75.0,6.8,1332.0,sleep|magic|prince|castle|fairy tale|kingdom|m...,Mary Costa,Bill Shirley,Eleanor Audley,Clyde Geronimi
7296,False,31000000,Family|Fantasy|Comedy,14442,en,Ella Enchanted,11.316988,2004.0,27388767.0,96.0,5.9,454.0,elves|based on novel|magic|fairy|prince|fairy ...,Anne Hathaway,Hugh Dancy,Cary Elwes,Tommy O'Haver


In [147]:
movies.iloc[suggest_me_movies('the mummy', 30, remove_sequel=True),:]

your selection:
adult                                                            False
budget                                                        80000000
genres                                        Adventure|Action|Fantasy
id                                                                 564
original_language                                                   en
original_title                                               The Mummy
popularity                                                     23.9841
release_date                                                      1999
revenue                                                    4.15885e+08
runtime                                                            124
vote_average                                                       6.6
vote_count                                                        2796
plot_keywords        library|secret passage|cairo|egypt|pastor|pyra...
actor_1_name                                            Brend

,adult,budget,genres,id,original_language,original_title,popularity,release_date,revenue,runtime,vote_average,vote_count,plot_keywords,actor_1_name,actor_2_name,actor_3_name,director_name
1177,False,18000000,Adventure|Action,85,en,Raiders of the Lost Ark,19.901576,1981.0,389925971.0,115.0,7.7,3949.0,saving the world|riddle|nepal|himalaya|cairo|m...,Harrison Ford,Karen Allen,Paul Freeman,Steven Spielberg
1267,False,48000000,Adventure|Action,89,en,Indiana Jones and the Last Crusade,14.788987,1989.0,474171806.0,127.0,7.6,3221.0,saving the world|venice|holy grail|library|rid...,Harrison Ford,Sean Connery,Denholm Elliott,Steven Spielberg
20923,False,17000000,Horror,109428,en,Evil Dead,12.015250,2013.0,97542952.0,91.0,6.4,1754.0,remake|demon|necronomicon|duringcreditsstinger,Jane Levy,Shiloh Fernandez,Lou Taylor Pucci,Fede Alvarez
37314,False,140000000,Fantasy,205584,en,Gods of Egypt,13.337707,2016.0,150680864.0,127.0,5.2,1303.0,egypt|underworld|fight|mythology|nile|war|thie...,Brenton Thwaites,Nikolaj Coster-Waldau,Gerard Butler,Alex Proyas
37315,False,140000000,Fantasy,205584,en,Gods of Egypt,13.337707,2016.0,150680864.0,127.0,5.2,1303.0,egypt|underworld|fight|mythology|nile|war|thie...,Brenton Thwaites,Nikolaj Coster-Waldau,Gerard Butler,Alex Proyas
1183,False,15000000,Adventure|Drama|History|War,947,en,Lawrence of Arabia,10.893332,1962.0,69995385.0,216.0,7.8,870.0,cairo|arabian|world war i|horse|jerusalem|brit...,Peter O'Toole,Alec Guinness,Anthony Quinn,David Lean
23176,False,145000000,Animation|Adventure|Family,82703,en,Mr. Peabody & Sherman,9.493838,2014.0,272912430.0,92.0,6.7,861.0,father son relationship|egypt|intelligence|ado...,Ty Burrell,Max Charles,Ariel Winter,Rob Minkoff
2301,False,70000000,Adventure|Animation|Drama|Family|Music,9837,en,The Prince of Egypt,11.898185,1998.0,218613188.0,99.0,6.9,847.0,moses|egypt|pyramid|exodus|kingdom|governance|...,Val Kilmer,Ralph Fiennes,Patrick Stewart,Brenda Chapman
2374,False,20000000,Drama|Thriller|Mystery,192,en,The Name of the Rose,11.469751,1986.0,77200000.0,130.0,7.2,739.0,christianity|library|inquisition|monk|poison|s...,Sean Connery,Christian Slater,Helmut Qualtinger,Jean-Jacques Annaud
8273,False,32000000,Music|Adventure|Animation|Action|Comedy,3989,en,Team America: World Police,11.428941,2004.0,50907422.0,98.0,6.6,667.0,paris|france|cairo|capitalism|loss of lover|eg...,Trey Parker,Matt Stone,Kristen Miller,Trey Parker
